# A basic `TensorFlow` installation check

Pre-reqs: If nothing is working right, check this notebook, [../jims_ml_notebook_env.py].

A sanity check call to `tensorflow` to make sure it finds and loads the library. NOTE: this does not do any tensor processing:

In [ ]:
import tensorflow as tf

print(f"A TF computation: {tf.add(1, 5).numpy()}")

hello = tf.constant('Hello, TensorFlow!')
print(f"A TensorFlow string constant: {hello.numpy()}")

# Of special `import` for `keras`
Keras notions should be imported from `keras.*`, not from `tensorflow`.

When coding to the conventional API:

In [ ]:

# Imports when using the conventional class-centric Keras API

# from keras.api.datasets import mnist
from keras.api.models import Sequential
from keras.api.layers import Input, Dense




When coding to the newer functional Keras API:


In [ ]:
import keras  # has keras.Model keras.Input etc


Now we load a dataset from the convenient tensorflow_datasets python package.

In [ ]:
import importlib
import jkcsoft.ml.tfds_utils

importlib.reload(jkcsoft.ml.tfds_utils)

from jkcsoft.ml.tfds_utils import dump_dataset_info, display_dataset_info


# Validates datasets package install and lists voluminous available datasets
dump_dataset_info()

# Create an array of 5 most popular datasets
most_popular_datasets = ['mnist', 'cifar10', 'imdb_reviews', 'fashion_mnist', 'coco']
print(f"Most popular datasets: {most_popular_datasets}")

for dataset_name in most_popular_datasets:
    display_dataset_info(dataset_name)


# Setup training and test datums to be used in the next two examples

The common datums are used to compare the use of the two Keras APIs.


In [ ]:
from jkcsoft.ml.sandbox.tf_tinker import ModelTest, RunBatch

# define a batch include a common dataset for all tests
run_batch = RunBatch()
run_batch.x_train = tf.constant([[1.0], [2.0], [3.0], [4.0]])
run_batch.y_train = tf.constant([[2.0], [4.0], [6.0], [8.0]])
run_batch.x_test = tf.constant([[5.0], [6.0]])
run_batch.y_test = tf.constant([[10.0], [12.0]])

# A simple _conventional_ Keras pipeline with hard-coded TensorFlow datasets

In [ ]:
# Define a model using the conventional Keras class API
model = Sequential([
    Input(shape=(1,)),  # Input layer for single-dimensional input
    Dense(32, activation="relu"),  # kera.src.core.dense.Dense
    Dense(16, activation="sigmoid"),
    Dense(16, activation="softmax"),
    Dense(1, activation="linear")
])

test_common_dense = ModelTest().set_description("Common Dense Sequence").set_model(model)
test_common_dense.set_fit_epochs(10)

# invoke model compile here
test_common_dense.compile_model(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        "mean_absolute_error",
        "mean_absolute_percentage_error",
        "mean_squared_error",
        "root_mean_squared_error",
        "accuracy",
        "precision",
        "recall",
        "auc",
        "binary_accuracy"
    ])

# test_common_dense.model.fit()

run_batch.add_test(test_common_dense)

# run all models against the same batch dataset
run_batch.run_all()

print('\nTest accuracy:', run_batch.results[0].eval_results)

# A simple pipeline with the Keras _functional_ API

In [ ]:
from keras import layers

keras.backend.clear_session()

# Keep the input tensor
inputs_fn = keras.Input(shape=(1,))

print(f"inputs shape: {inputs_fn.shape}")

print(f"inputs dtype: {inputs_fn.dtype}")

# pipe input
d1 = layers.Dense(32, activation="relu")
x = d1(inputs_fn)

# pipe thru proc layers
d2 = layers.Dense(16, activation="sigmoid")
x = d2(x)
d3 = layers.Dense(8, activation="softmax")
x = d3(x)

# use final layer to reduce to output
final_layer = layers.Dense(1, activation="linear")
outputs_fn = final_layer(x)

functional_model = keras.Model(inputs=inputs_fn, outputs=outputs_fn, name="functional_model_1")

print(f"functional_model summary: {functional_model.summary()}")

functional_model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        "mean_absolute_error",
        "mean_absolute_percentage_error",
        "mean_squared_error",
        "root_mean_squared_error",
        "accuracy",
        "precision",
        "recall",
        "auc",
        "binary_accuracy"
    ]
)

functional_model.fit(run_batch.x_train, run_batch.y_train, epochs=3, verbose=1)

loss_and_metrics = functional_model.evaluate(run_batch.x_test, run_batch.y_test, verbose=2)

print(f"Test eval results: {loss_and_metrics}")

## A more readable functional pipeline declaration

In [ ]:
from jkcsoft.ml import keras_utils

importlib.reload(keras_utils)

proc_layers = [d1, d2, d3, final_layer]

composite_model = keras_utils.compose_layers(inputs_fn, proc_layers, verbose=False)

print(f"composite_model summary: {composite_model.summary()}")

composite_model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
#        "mean_absolute_error",
        "mean_absolute_percentage_error",
#        "mean_squared_error",
#        "root_mean_squared_error",
#        "accuracy",
#        "precision",
#        "recall",
#        "auc",
#        "binary_accuracy"
    ]
)

composite_model.fit(run_batch.x_train, run_batch.y_train, epochs=3, verbose=1)

loss_and_metrics = composite_model.evaluate(run_batch.x_test, run_batch.y_test, verbose=2)

print(f"Test eval results: {loss_and_metrics}")

## A _functional_ model loaded into our `RunBatch` apparatus

In [ ]:
keras.backend.clear_session()

test_common_dense_fn = ModelTest().set_description("Common Dense Sequence").set_model(functional_model)
test_common_dense_fn.set_fit_epochs(10)
test_common_dense_fn.compile_model(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
#        "mean_absolute_error",
        "mean_absolute_percentage_error",
#        "mean_squared_error",
#        "root_mean_squared_error",
#        "accuracy",
#        "precision",
#        "recall",
#        "auc",
#        "binary_accuracy"
    ]
)

test_common_dense_fn.model.summary()

test_common_dense_fn.model.fit(run_batch.x_train, run_batch.y_train, epochs=10, verbose=1)

run_batch.add_test(test_common_dense_fn)

run_batch.run_all()

for i, result in enumerate(run_batch.results):
    print(f"Test {i} eval results: {result.eval_results}")
